Overview the file structure:

In [1]:
import tensorflow as tf
import numpy as np
from tabulate import tabulate

def inspect_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    tensor_details = interpreter.get_tensor_details()
    ops = interpreter._get_ops_details()

    print("\n" + "="*50)
    print(f"Model Inspection: {model_path}")
    print("="*50 + "\n")

    # 1. Print Input/Output Details
    print("\n[Input Tensors]")
    input_table = []
    for i, detail in enumerate(input_details):
        input_table.append([
            i, detail['name'], detail['shape'], detail['dtype'],
            detail.get('quantization', 'N/A')
        ])
    print(tabulate(input_table, headers=["Index", "Name", "Shape", "DType", "Quantization"]))

    print("\n[Output Tensors]")
    output_table = []
    for i, detail in enumerate(output_details):
        output_table.append([
            i, detail['name'], detail['shape'], detail['dtype'],
            detail.get('quantization', 'N/A')
        ])
    print(tabulate(output_table, headers=["Index", "Name", "Shape", "DType", "Quantization"]))

    print("\n[All Tensors]")
    tensor_table = []
    for tensor in tensor_details:
        tensor_table.append([
            tensor['index'], tensor['name'], tensor['shape'], tensor['dtype'],
            tensor.get('quantization', 'N/A'), tensor.get('sparsity_parameters', 'N/A')
        ])
    print(tabulate(tensor_table, headers=["Index", "Name", "Shape", "DType", "Quantization", "Sparsity"]))

    print("\n[Operations]")
    ops_table = []
    for op in ops:
        ops_table.append([
            op['index'], op['op_name'], op['inputs'], op['outputs']
        ])
    print(tabulate(ops_table, headers=["Index", "Op Name", "Inputs", "Outputs"]))

    print("\n[Model Summary]")
    print(f"- Inputs: {len(input_details)}")
    print(f"- Outputs: {len(output_details)}")
    print(f"- Tensors: {len(tensor_details)}")
    print(f"- Operations: {len(ops)}")

if __name__ == "__main__":
    model_path = "d-vector-extractor-256.tflite"
    inspect_tflite_model(model_path)


Model Inspection: d-vector-extractor-256.tflite


[Input Tensors]
  Index  Name                     Shape          DType                    Quantization
-------  -----------------------  -------------  -----------------------  --------------
      0  serving_default_input:0  [ 1 40 40  1]  <class 'numpy.float32'>  (0.0, 0)

[Output Tensors]
  Index  Name                         Shape      DType                    Quantization
-------  ---------------------------  ---------  -----------------------  --------------
      0  StatefulPartitionedCall_1:0  [  1 256]  <class 'numpy.float32'>  (0.0, 0)

[All Tensors]
  Index  Name                                                                                                                                                                              Shape          DType                    Quantization    Sparsity
-------  -------------------------------------------------------------------------------------------------------------------------

Code to extract .npy files for each tensor according to the model previously generated

In [2]:
import numpy as np
import tensorflow as tf
import os

model_path = 'd-vector-extractor-256.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

output_dir = "tflite_tensors"
os.makedirs(output_dir, exist_ok=True)
tensor_details = interpreter.get_tensor_details()

for tensor in tensor_details:
    try:
        if interpreter.get_tensor(tensor['index']).size == 0:
            print(f"Skipping empty tensor: {tensor['name']} (index {tensor['index']})")
            continue

        tensor_data = interpreter.get_tensor(tensor['index'])
        tensor_name = tensor['name'].replace('/', '_')

        filename = f"tensor_{tensor['index']:03d}_{tensor_name}_shape-{tensor['shape']}_dtype-{tensor['dtype']}.npy"
        np.save(os.path.join(output_dir, filename), tensor_data)
        print(f"Saved: {filename}")

    except ValueError as e:
        print(f"Failed to extract {tensor['name']} (index {tensor['index']}): {str(e)}")
        continue

print(f"\nExtraction complete. Output directory: {output_dir}/")


Saved: tensor_000_serving_default_input:0_shape-[ 1 40 40  1]_dtype-<class 'numpy.float32'>.npy
Saved: tensor_001_arith.constant_shape-[64]_dtype-<class 'numpy.float32'>.npy
Saved: tensor_002_arith.constant1_shape-[32]_dtype-<class 'numpy.float32'>.npy
Saved: tensor_003_arith.constant2_shape-[16]_dtype-<class 'numpy.float32'>.npy
Saved: tensor_004_arith.constant3_shape-[8]_dtype-<class 'numpy.float32'>.npy
Saved: tensor_005_arith.constant4_shape-[64  3  3 32]_dtype-<class 'numpy.float32'>.npy
Saved: tensor_006_arith.constant5_shape-[32  3  3 16]_dtype-<class 'numpy.float32'>.npy
Saved: tensor_007_arith.constant6_shape-[16  3  3  8]_dtype-<class 'numpy.float32'>.npy
Saved: tensor_008_arith.constant7_shape-[8 3 3 1]_dtype-<class 'numpy.float32'>.npy
Saved: tensor_009_arith.constant8_shape-[2]_dtype-<class 'numpy.int32'>.npy
Saved: tensor_010_d-vector-extractor-256_1_batch_normalization_1_batchnorm_mul_shape-[1]_dtype-<class 'numpy.float32'>.npy
Saved: tensor_011_d-vector-extractor-256_1_

Code to verify content of .npy files generated:

In [3]:
import numpy as np
import os
import matplotlib.pyplot as plt

def verify_npy_files(directory="tflite_tensors"):
    """Verify contents of .npy files in a directory."""
    npy_files = [f for f in os.listdir(directory) if f.endswith('.npy')]

    if not npy_files:
        print(f"No .npy files found in {directory}!")
        return

    print(f"Found {len(npy_files)} .npy files in {directory}:\n")

    for file in sorted(npy_files):
        filepath = os.path.join(directory, file)
        data = np.load(filepath)

        print(f"{file}:")
        print(f"  - Shape: {data.shape}")
        print(f"  - Dtype: {data.dtype}")
        print(f"  - Min: {np.min(data):.4f}, Max: {np.max(data):.4f}, Mean: {np.mean(data):.4f}")
        print(f"  - Size: {data.size} elements\n")

        if len(data.shape) == 4 and 'conv' in file.lower():
            print("  Visualizing first filter of 4D weights...")
            plt.figure(figsize=(3, 3))
            plt.imshow(data[0, :, :, 0], cmap='viridis')
            plt.colorbar()
            plt.title(f"{file} (first filter)")
            plt.show()

if __name__ == "__main__":
    verify_npy_files()


Found 13 .npy files in tflite_tensors:

tensor_000_serving_default_input:0_shape-[ 1 40 40  1]_dtype-<class 'numpy.float32'>.npy:
  - Shape: (1, 40, 40, 1)
  - Dtype: float32
  - Min: -2408881504095047458328936448.0000, Max: 501704745483963226652672.0000, Mean: -3009670458895549645979648.0000
  - Size: 1600 elements

tensor_001_arith.constant_shape-[64]_dtype-<class 'numpy.float32'>.npy:
  - Shape: (64,)
  - Dtype: float32
  - Min: -0.4863, Max: 0.6305, Mean: -0.0099
  - Size: 64 elements

tensor_002_arith.constant1_shape-[32]_dtype-<class 'numpy.float32'>.npy:
  - Shape: (32,)
  - Dtype: float32
  - Min: -0.9408, Max: 0.5785, Mean: -0.0547
  - Size: 32 elements

tensor_003_arith.constant2_shape-[16]_dtype-<class 'numpy.float32'>.npy:
  - Shape: (16,)
  - Dtype: float32
  - Min: -0.7575, Max: 0.4480, Mean: -0.1527
  - Size: 16 elements

tensor_004_arith.constant3_shape-[8]_dtype-<class 'numpy.float32'>.npy:
  - Shape: (8,)
  - Dtype: float32
  - Min: -0.4341, Max: 0.7258, Mean: 0.1544


Deployment of header file of d_vector_extractor.h

In [9]:
import numpy as np
import os

def sanitize_name(name):
    """Convert tensor names to valid C identifiers."""
    return name.replace('/', '_').replace(';', '_').replace(':', '_')

def estimate_name(name):
  match name:
    case "serving_default_input_0":
        return "default_input"
    case "arith.constant":
        return "conv_4_BiasAdd_ReadVariableOp"
    case "arith.constant1":
        return "conv_3_BiasAdd_ReadVariableOp"
    case "arith.constant2":
        return "conv_2_BiasAdd_ReadVariableOp"
    case "arith.constant3":
        return "conv_1_BiasAdd_ReadVariableOp"
    case "arith.constant4":
        return "conv_4_Weights"
    case "arith.constant5":
        return "conv_3_Weights"
    case "arith.constant6":
        return "conv_2_Weights"
    case "arith.constant7":
        return "conv_1_Weights"
    case "arith.constant8":
        return "reshape_Dimensions"
    case "d-vector-extractor-256_1_batch_normalization_1_batchnorm_mul":
        return "batch_norm_mul"
    case "d-vector-extractor-256_1_batch_normalization_1_batchnorm_sub":
        return "batch_norm_sub"

def generate_header(npy_dir="tflite_tensors", output_file="d_vector_extractor.h"):
    """Generate a C header from .npy files."""
    npy_files = [f for f in os.listdir(npy_dir) if f.endswith('.npy')]

    with open(output_file, "w") as f:
        f.write("#ifndef D_VECTOR_EXTRACTOR_H\n")
        f.write("#define D_VECTOR_EXTRACTOR_H\n\n")

        for file in sorted(npy_files):
            data = np.load(os.path.join(npy_dir, file))
            if data.size == 0:
                continue

            tensor_name = "_".join(file.split('_')[2:-2])
            c_name = sanitize_name(tensor_name)
            c_name = estimate_name(c_name)
            f.write(f"static const float {c_name}[{data.size}] = {{\n")

            flat_data = data.flatten()
            for i in range(0, len(flat_data), 8):
                line = ", ".join(f"{x:.6f}f" for x in flat_data[i:i+8])
                f.write(f"    {line},\n")

            f.write("};\n\n")
            f.write(f"// Shape: {data.shape}\n\n")
        f.write("#endif // D_VECTOR_EXTRACTOR_H\n")

if __name__ == "__main__":
    generate_header()